# Anwendung 1: Keyword Extraction

Eine mögliche Anwendung von Textanalysen auf der Grundlage eines Vector-Space-Modells ist die Identifikation von Schlüsselwörtern für Texte, die »Keyword Extraction«. Ein Schlüsselwort kann verstanden werden als ein Wort, dass einen inhaltlichen Aspekt eines Textes beschreibt. Eine begrenzte Anzahl von Schlüsselwörtern sollte also bereits einen guten Eindruck vom Inhalt eines Textes vermitteln. Schlagwortsysteme, etwa in Bibliothekskatalogen oder Literaturdatenbanken, bauen auf dieser Idee auf. Sie erfordern aber eine sorgfältige manuelle Verschlagwortung. Daher ist die Frage interessant, ob sich Schlüsselwörter für Texte auch automatisch aus dem Textinhalt erschließen lassen.

Ein erster Kandidat für die Identifikation wäre die reine Worthäufigkeit: Für einen Text relevante Wörter sollten in ihm häufig vorkommen. Die Häufigkeitsauszählungen aus den vorangegangenen Einheiten zeigen aber, dass eine reine Häufigkeitsliste nicht sehr hilfreich ist: Zunächst erscheinen Funktionwörter wie etwa Artikel ganz oben in der Liste. Aber auch wenn man nach Wortarten filtert, bleiben oft relativ unspezifische Wörter dominant.

Der Keyword Extraction liegt daher die Annahme zugrunde, dass einige Wörter insgesamt häufig vorkommen, ohne dass sie inhaltlich aussagekräftig wären. Ihr häufiges Vorkommen ist daher eher charakteristisch für eine (Fach-)Sprache als für einzelne Texte. Die Verteilung dieser Wörter sollte entsprechend über die Texte relativ gleich bleiben. Ein Schlüsselwort dagegen sollte im entsprechenden Text deutlich häufiger vorkommen als im Gesamtkorpus. Statistisch lässt sich ein Schlüsselwort also definieren als ein Wort, das in einem einzelnen Text signifikant häufiger vorkommt als seine Häufigkeit im Gesamtcorpus erwarten lassen würde.

Eine Form einer entsprechenden Identifikation von Schlüsselwörtern ist [Tf-Idf](http://de.wikipedia.org/wiki/Tf-idf-Ma%C3%9F). Das Kürzel steht dabei für »Termfrequenz – Inverse Dokumentfrequenz«.

Dieses Verfahren ist in gensim implementiert. Dafür wird die in der vorangegangenen Einheit vorgestellte Corpus-Klasse verwendet.

In [1]:
from gensim.corpora.textcorpus import TextCorpus
from textblob_de import TextBlobDE as TextBlob
import pandas as pd

class CSVCorpus(TextCorpus):
    """Read corpus from a csv file."""

    def get_texts(self):
        with self.getstream() as csvfile:  # Öffnet die CSV-Datei
            table = pd.read_csv(csvfile, parse_dates=['date'], encoding='utf-8')  # Liest die CSV-Datei
            for text in table['text']:  # Verarbeite die einzelnen Texte aus der Spalte 'text'
                blob = TextBlob(text)
                yield blob.words

Das Vorgehen in gensim ist dabei nicht ganz selbsterklären, daher hier eine kurze Erläuterung: Der Befehl `TfidfModel(corpus)` erzeugt zunächst ein Modell, das die allgemeine Häufigkeitsverteilung von Wörtern im Corpus enthält. Mit diesem Modell können nun einzelne Dokumente, aber auch nur einzelne Wörter aus einem Dokument, verglichen werden. Dies ist etwa dann sinnvoll, wenn eine Berechnung für das gesamte Corpus zu aufwändig wäre, oder wenn neue Texte hinzukommen, die mit einem bestehenden Corpus verglichen werden sollen.

Um dagegen für das gesamte Corpus die Tf-Idf-Werte zu berechnen, wird dem Modell im zweiten Schritt einfach wiederum das gesamte Corpus übergeben: `tf_idf[corpus]`. In diesem Schritt findet dann die eigentliche Berechnung für die einzelnen Dokument statt.

In [2]:
from gensim.models import TfidfModel

corpus = CSVCorpus("../Daten/reden.csv")
tf_idf = TfidfModel(corpus)
corpus_idf = tf_idf[corpus]

Dies gibt für jedes Wort in jedem Dokument einen Wert aus. Das Format ist dabei das gleiche wie für Corpora, nur dass anstelle der reinen Worthäufigkeiten der tfidf-Wert ausgegeben wird. Ein Dokument ist also eine Liste aus `(wortindex, wert)`-Paaren.

Um nun für einzelne Dokumente die Schlüsselwörter zu bestimmen, ist es sinnvoll, die Wörter im Dokument nach ihrem tfidf-Wert zu sortieren. Zugleich interessieren nicht die numerischen Wort-Indices, sondern die eigentlichen Wörter. Daher wird hier eine Funktion `keywords` definiert, die das Dokument zunächst nach tfidf-Wert (dem zweiten Eintrag im Wort-Wert-Paar: `x[1]`) sortiert und dann ein Paar aus Wörterbucheintrag `corpus.dictionary[term]` und tfidf-Wert ausgibt.

In [3]:
def keywords(corpus, doc_tf_idf):
    return [(corpus.dictionary[term], value)
            for term, value
            in sorted(doc_tf_idf, key=lambda x: x[1], reverse=True)]

Für die ersten zehn Dokumente können nun etwa jeweils fünf Schlüsselwörter ausgegeben werden:

In [4]:
for i, doc in enumerate(corpus_idf):
    if i == 9:
        break
    print
    for keyword, value in keywords(corpus, doc)[0:5]:
        print u'  {} ({})'.format(keyword, value)


  Safranski (0.472750464052)
  Rüdiger (0.337367236021)
  Wahrheiten (0.138215543768)
  Utopien (0.126066790414)
  Romantik (0.104493794377)

  „Elektromobilität“ (0.372779168826)
  Stecker (0.372435485485)
  Elektromobilität (0.173383544659)
  Präsentation (0.171540045793)
  China (0.162105143183)

  Nachhaltigkeit (0.294015094791)
  China (0.291787735475)
  Universität (0.109320088669)
  Innovationspartnerschaft (0.1065077783)
  Chinas (0.102461803818)

  Tempelhof (0.556694789109)
  Flughafen (0.292116704843)
  AlliiertenMuseum (0.209741456143)
  Berlin-Tempelhof (0.151825851575)
  Miniaturbild (0.151825851575)

  „Sonne“ (0.446115651832)
  Schiff (0.272536385343)
  Neptun (0.227323234857)
  Werft (0.213883738352)
  Meeresforschung (0.188422877129)

  Klimaschutz (0.192756524708)
  Hendricks (0.152513059156)
  Umweltminister (0.14994348321)
  Energien (0.13244948829)
  2030 (0.125330990937)

  Bevölkerungsschutz (0.211489490766)
  Bevölkerungsschutzes (0.211489490766)
  Haupt (0.18

Hier ist auffällig, dass fast nur Substantive ausgegeben werden, obwohl das Corpus selbst die Wörter überhaupt nicht gefiltert hat. Dies legt den Schluss nahe, dass andere Wortarten deutlich gleicher über das Corpus verteilt sind.

Um die Plausibilität dieser Listen zu überprüfen, können zusätzlich die Titel der Dokumente ausgegeben werden:

In [5]:
data = pd.read_csv("../Daten/reden.csv", parse_dates=['date'], encoding='utf-8')

for i, doc in enumerate(corpus_idf):
    if i == 9:
        break
    print
    print data['title'][i]
    for keyword, value in keywords(corpus, doc)[0:5]:
        print u'  {} ({})'.format(keyword, value)


Laudatio auf Rüdiger Safranski
  Safranski (0.472750464052)
  Rüdiger (0.337367236021)
  Wahrheiten (0.138215543768)
  Utopien (0.126066790414)
  Romantik (0.104493794377)

Rede von Bundeskanzlerin Merkel anlässlich der Präsentation des deutsch-chinesischen Kooperationsprojekts zur Ladeinfrastruktur für Elektrofahrzeuge am 8. Juli 2014
  „Elektromobilität“ (0.372779168826)
  Stecker (0.372435485485)
  Elektromobilität (0.173383544659)
  Präsentation (0.171540045793)
  China (0.162105143183)

Rede von Bundeskanzlerin Merkel anlässlich des Besuchs der Tsinghua - Universität am 8. Juli 2014
  Nachhaltigkeit (0.294015094791)
  China (0.291787735475)
  Universität (0.109320088669)
  Innovationspartnerschaft (0.1065077783)
  Chinas (0.102461803818)

Kulturstaatsministerin Monika Grütters zur Eröffnung der Sonderausstellung
"Flughafen Berlin-Tempelhof - Die amerikanische Geschichte"
  Tempelhof (0.556694789109)
  Flughafen (0.292116704843)
  AlliiertenMuseum (0.209741456143)
  Berlin-Tempelh